## Creating the environement

In [ ]:
import gymnasium as gym
import pygame
import numpy as np
from IPython.display import clear_output
from IPython import display
import matplotlib.pylab as plt
import statistics
from env import SimpleCorridor

## Test Agent Function 

In [ ]:
def test_agent(env: gym.Env, policy: callable, episodes: int = 10) -> None: 
    for episode in range(episodes):
        temp = env.reset()
        tagent_pos = temp[0]
        ttarget_pos = temp[1]
        done = False  
       
        env.render(mode="rgb_array")
        while not done:
            action = policy(tagent_pos, 0.1) 

            next_state, _, done, _ = env.step(action)
            env.render()
            plt.axis('off')
            display.display(plt.gcf())
            display.clear_output(wait=True)
            
            
            tagent_pos = next_state[0]

## Initialising the environment 

In [ ]:
env = SimpleCorridor()
env.reset()
env.render()


## Q Value Table

In [ ]:
action_values = np.zeros(shape=(15,2))
print(action_values)

### Creating the Policy

In [ ]:
def policy(state, epsilon = 0.1):
    if np.random.random() < epsilon:
        return np.random.choice(2)
    else: 
        av = action_values[state]
        return np.random.choice(np.flatnonzero(av == av.max()))

In [ ]:
action = policy(0, epsilon = 0.5)
print(f"The action taken in state 0 is {action}")

#### Testing the policy

# Implementing the algorithm

In [ ]:
def on_policy_mc_control(policy, action_values, episodes=1000, epsilon=0.1, gamma=0.99):
    sa_returns = np.empty((15, 2), dtype=object)

    for episode in range(0, episodes + 1):
        state, target = env.reset()
        done = False
        transitions = []

        while not done:
            action = policy(state, epsilon)
            (next_state, target), reward, done, info = env.step(action)
            transitions.append([state, action, reward])
            state = next_state

        G = 0

        for state_t, action_t, reward_t in reversed(transitions):
            G = reward_t + gamma * G
            if sa_returns[state_t, action_t] == None:
                sa_returns[state_t, action_t] = [G]
            else:
                sa_returns[state_t, action_t].append(G)
            action_values[state_t, action_t] = statistics.mean(
                sa_returns[state_t, action_t])


In [ ]:
on_policy_mc_control(policy, action_values)

In [ ]:
test_agent(env, policy, episodes=10)